Импортируем библиотеки, грузим датасеты

In [43]:
# Импортируем библиотеки

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

# Грузим датасеты

df_train = pd.read_csv('data/train_processed.csv', low_memory=False)
df_valid = pd.read_csv('data/valid_processed.csv', low_memory=False)

Baseline модель

In [45]:
# Разделяем датасет на таргет и признаки
X = df_train.drop(columns=['target', 'report_date', 'col1454'])
y = df_train['target']

X_valid = df_valid.drop(columns=['target', 'report_date', 'col1454'])

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Модель
baseline_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Обучаем модель
baseline_model.fit(X_train, y_train)

# Предсказание вероятностей
y_pred_proba = baseline_model.predict_proba(X_test)[:, 1]

# Метрика модели на train датасете
roc_auc = roc_auc_score(y_test, y_pred_proba)
y_pred = baseline_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")

ROC-AUC: 0.9715
Accuracy: 0.9851


In [46]:
y_test_pred_proba = baseline_model.predict_proba(X_valid)[:, 1]

valid_accuracy = accuracy_score(y_test, y_pred)